# TESTE DE NIVELAMENTO
Gilson Dias

## 1. Teste de WEB SCRAPING

In [7]:
import requests
from bs4 import BeautifulSoup
import os
import zipfile

# URL principal da página da ANS
url_base = "https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos"

# Criar uma pasta para armazenar os PDFs
pasta_downloads = "anexos"
os.makedirs(pasta_downloads, exist_ok=True)

# Analisar o HTML da página
soup = BeautifulSoup(resposta.text, "html.parser")

# Procurar todos os links na página
links = soup.find_all("a", href=True)

# Filtrar apenas os links dos PDFs dos anexos I e II
pdf_links = [
    link["href"] for link in links
    if ("Anexo_I" in link["href"] or "Anexo_II" in link["href"]) and link["href"].endswith(".pdf")
]

# Ajustar links relativos (se houver)
pdf_links = [
    link if link.startswith("http") else f"https://www.gov.br{link}" for link in pdf_links
]

# Lista para armazenar os arquivos baixados
arquivos_baixados = []

# Baixar os PDFs encontrados
for pdf_link in pdf_links:
    nome_arquivo = pdf_link.split("/")[-1]  # Nome do arquivo PDF
    caminho_arquivo = os.path.join(pasta_downloads, nome_arquivo)

    # Fazer download do PDF
    resposta_pdf = requests.get(pdf_link, stream=True)
    if resposta_pdf.status_code == 200:
        with open(caminho_arquivo, "wb") as f:
            for chunk in resposta_pdf.iter_content(chunk_size=1024):
                f.write(chunk)
        print(f"✔ Arquivo baixado: {nome_arquivo}")
        arquivos_baixados.append(caminho_arquivo)
    else:
        print(f"❌ Erro ao baixar: {nome_arquivo} (status {resposta_pdf.status_code})")

# Criar um arquivo ZIP contendo os PDFs baixados
arquivo_zip = "anexos.zip"
with zipfile.ZipFile(arquivo_zip, "w") as zipf:
    for arquivo in arquivos_baixados:
        zipf.write(arquivo, os.path.basename(arquivo))

print(f"📂 Todos os arquivos foram compactados em: {arquivo_zip}")


✔ Arquivo baixado: Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
✔ Arquivo baixado: Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf
📂 Todos os arquivos foram compactados em: anexos.zip


## 2. Teste de transformação de dados

##### 2.1 Extraindo os dados do Anexo 1

In [ ]:
pip install pdfplumber pandas


In [85]:
import pandas as pd
import pdfplumber

# Caminho do PDF
arquivo_pdf = "./anexos/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf"

tabelas = []

# Abrir o PDF e processar todas as páginas
with pdfplumber.open(arquivo_pdf) as pdf:
    for pagina in pdf.pages:
        tabela = pagina.extract_table()
        if tabela:
            tabelas.append(pd.DataFrame(tabela))

# Unir todas as tabelas extraídas
df = pd.concat(tabelas, ignore_index=True)

df.head(5)




,0,1,2,3,4,5,6,7,8,9,10,11,12
0,PROCEDIMENTO,RN\n(alteração),VIGÊNCIA,OD,AMB,HCO,HSO,REF,PAC,DUT,SUBGRUPO,GRUPO,CAPÍTULO
1,ACONSELHAMENTO GENÉTICO,,,,AMB,HCO,HSO,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
2,ATENDIMENTO/ACOMPANHAMENTO EM HOSPITAL-DIA PSI...,,,,,HCO,HSO,REF,,109,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
3,ATENDIMENTO INTEGRAL AO RECÉM-NASCIDO (SALA DE...,,,,,HCO,,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
4,ATENDIMENTO MÉDICO DO INTENSIVISTA EM UTI GERA...,,,,,HCO,HSO,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS


##### 2.2 Salvando os dados em uma tabela estruturada, em formato csv

In [75]:
arquivo_csv = "tabela_extraida.csv"

df.to_csv(arquivo_csv, index=False, encoding="utf-8")

print("Tabelas extraídas e salvas em 'tabelas_extraidas.csv'!")

Tabelas extraídas e salvas em 'tabelas_extraidas.csv'!


##### 2.3 Compactando o csv em um arquivo denominado "Teste_Gilson.zip"

In [76]:
arquivo_zip = "Teste_Gilson.zip"

with zipfile.ZipFile(arquivo_zip, "w") as zipf:
    zipf.write(arquivo_csv, os.path.basename(arquivo_csv))

print("Arquivo csv compactado!")

Arquivo csv compactado!


##### 2.4 . Substituindo as abreviações das colunas OD e AMB pelas descrições completas, conforme a legenda no rodapé.

In [ ]:
# Definir a segunda linha como cabeçalho (substituindo a primeira)
df.columns = df.iloc[0] 
df = df[1:].reset_index(drop=True)  

df.rename(columns={"OD": "Seg. Odontológica", "AMB": "Seg. Ambulatorial"}, inplace=True)
df = df.replace({"OD": "Seg. Odontológica", "AMB": "Seg. Ambulatorial"}, regex=True)

df.head(10)

,PROCEDIMENTO,RN\n(alteração),VIGÊNCIA,Odontológica,Seg. Ambulatorial,HCO,HSO,REF,PAC,DUT,SUBGRUPO,GRUPO,CAPÍTULO
0,ACONSELHAMENTO GENÉTICO,,,,Seg. Ambulatorial,HCO,HSO,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
1,ATENDIMENTO/ACOMPANHAMENTO EM HOSPITAL-DIA PSI...,,,,,HCO,HSO,REF,,109,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
2,ATENDIMENTO INTEGRAL AO RECÉM-NASCIDO (SALA DE...,,,,,HCO,,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
3,ATENDIMENTO MÉDICO DO INTENSIVISTA EM UTI GERA...,,,,,HCO,HSO,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
4,ATENDIMENTO PEDIÁTRICO A GESTANTES (3º TRIMESTRE),,,,Seg. Ambulatorial,HCO,,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
5,ATIVIDADE EDUCACIONAL PARA PLANEJAMENTO FAMILIAR,,,,Seg. Ambulatorial,,,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
6,ATIVIDADE EDUCATIVA EM SAÚDE BUCAL,,,Seg. Odontológica,,,,,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
7,ATIVIDADE EDUCATIVA EM Seg. OdontológicaONTOLO...,,,Seg. Odontológica,,,,,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
8,CONDICIONAMENTO EM Seg. OdontológicaONTOLOGIA ...,,,Seg. Odontológica,,,,,,86,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
9,ESTABILIZAÇÃO DE PACIENTE POR MEIO DE CONTENÇÃ...,,,Seg. Odontológica,,,,,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS


## Testes 3 e 4

Os testes 3 e 4 não foram totalmente concluídos, pois ainda estou aprimorando minhas habilidades em SQL e no uso de APIs por meio da faculdade e cursos complementares. Continuo me desenvolvendo nessas áreas para melhorar minha execução dessas tarefas.